In [1]:
import tensorflow as tf

all_dataset = tf.data.Dataset.list_files(
    "/nfs/general/shared/Key*/*.jpg").shuffle(buffer_size=10000)

keys_tensor = tf.constant(['KeyA', 'KeyD'])
vals_tensor = tf.constant([0, 1])
init = tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor)
table = tf.lookup.StaticHashTable(init, default_value=-1)

def CreateImageAndLabel(filename):
    image = tf.io.read_file(filename)
    image = tf.io.decode_and_crop_jpeg(image, crop_window=tf.constant(
        [220, 260, 40, 80], dtype=tf.int32), channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    labelString = tf.strings.split(filename, '/')[4]
    # label = tf.one_hot(table.lookup(labelString),2)
    return image, table.lookup(labelString)


data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomTranslation(0.1, 0.1)
    ]
)

AUTOTUNE = tf.data.AUTOTUNE

test_dataset = all_dataset.take(1000).map(CreateImageAndLabel, num_parallel_calls=tf.data.AUTOTUNE,
                                          deterministic=False).batch(
    16).map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE,
            deterministic=False).cache().prefetch(buffer_size=AUTOTUNE)
train_dataset = all_dataset.skip(1000).repeat().map(CreateImageAndLabel, num_parallel_calls=tf.data.AUTOTUNE,
                                                    deterministic=False).batch(
    16).map(lambda x, y: (data_augmentation(x), y), num_parallel_calls=tf.data.AUTOTUNE,
            deterministic=False).prefetch(buffer_size=AUTOTUNE)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (5, 5), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(8, (3, 3), padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2)
])
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),  # 'adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])


tb_callback = tf.keras.callbacks.TensorBoard(
    '/tmp/logs', update_freq=1, histogram_freq=1, write_images=True)

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False)

# help(model.fit)

model.fit(train_dataset, epochs=100, steps_per_epoch=256,
          validation_data=test_dataset, callbacks=[tb_callback, model_checkpoint_callback])

# print(X.shape, y.shape)


2022-04-19 19:04:26.607500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 19:04:26.610981: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 19:04:26.611281: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-19 19:04:26.611724: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Epoch 1/100


2022-04-19 19:04:27.972392: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8400


 28/256 [==>...........................] - ETA: 0s - loss: 0.4687 - accuracy: 0.8616  

2022-04-19 19:04:28.220925: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


256/256 [==============================] - 2s 5ms/step - loss: 0.2513 - accuracy: 0.8960 - val_loss: 0.1496 - val_accuracy: 0.9540
Epoch 2/100
256/256 [==============================] - 1s 2ms/step - loss: 0.1469 - accuracy: 0.9592 - val_loss: 0.1185 - val_accuracy: 0.9890
Epoch 3/100
256/256 [==============================] - 1s 2ms/step - loss: 0.1171 - accuracy: 0.9744 - val_loss: 0.1166 - val_accuracy: 0.9780
Epoch 4/100
184/256 [====================>.........] - ETA: 0s - loss: 0.0950 - accuracy: 0.9854

KeyboardInterrupt: 

In [ ]:
tf.saved_model.save(model, '/nfs/general/shared/tf_models/color_classifier')
converter = tf.lite.TFLiteConverter.from_saved_model('/nfs/general/shared/tf_models/color_classifier') # path to the SavedModel directory
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
#   tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()
open('/nfs/general/shared/tflite/color_classifier.tflite', 'wb').write(tflite_model)